In [1]:
#!pip install transformers datasets evaluate sacrebleu
#!pip install 'huggingface_hub[tensorflow]'
#!pip install huggingface_hub
#!pip install tensorflow

In [2]:
import tensorflow as tf
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoTokenizer , DataCollatorForSeq2Seq , AdamWeightDecay , TFAutoModelForSeq2SeqLM
import evaluate
import numpy as np
from transformers import DataCollatorForSeq2Seq
import numpy as np
from transformers import AdamWeightDecay
from transformers import TFAutoModelForSeq2SeqLM
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback


In [5]:
notebook_login()

In [6]:
def load_dataset_huggin(argument):
  "Load the subset from datasets library"
  books = load_dataset("opus_books", argument)
  return books

In [7]:
english_french = load_dataset_huggin("en-fr")
english_hungarian = load_dataset_huggin("en-hu")
english_netherlands = load_dataset_huggin("en-nl")
english_russian = load_dataset_huggin("en-ru")
english_polish = load_dataset_huggin("en-pl")
english_italian = load_dataset_huggin("en-it")
# Tested quick
english_portuguese = load_dataset_huggin("en-pt")

Found cached dataset opus_books (/Users/gema/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset opus_books (/Users/gema/.cache/huggingface/datasets/opus_books/en-hu/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset opus_books (/Users/gema/.cache/huggingface/datasets/opus_books/en-nl/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


...

  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset opus_books (/Users/gema/.cache/huggingface/datasets/opus_books/en-it/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset opus_books (/Users/gema/.cache/huggingface/datasets/opus_books/en-pt/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
### We can select another languages to translate them, we based this on another languages that 
### have more rows so the finetunning makes more sense

In [9]:
def split_train_test(argument):
  "Load the subset from datasets library"
  books = argument["train"].train_test_split(test_size=0.2)
  return books

In [10]:
books = split_train_test(english_portuguese)

In [11]:
books["train"][0]

{'id': '126',
 'translation': {'en': "Her first idea was that she had somehow fallen into the sea, 'and in that case I can go back by railway,' she said to herself.",
  'pt': 'A sua primeira idéia era a de que, de algum modo, ela tinha caído no mar, "E, nesse caso, posso voltar de trem", falou para si mesma.'}}

In [12]:
checkpoint = "t5-small"

In [13]:

def load_tokenizer():
  "Load the tokenizer to process the language pairs"
  checkpoint = "t5-small"
  tokenizer = AutoTokenizer.from_pretrained(checkpoint , use_fast=False)
  return tokenizer

tokenizer = load_tokenizer()

In [14]:
source_lang = "en"
target_lang = "pt"
prefix = "translate English to French:"

In [15]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [16]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/1123 [00:00<?, ? examples/s]

Map:   0%|          | 0/281 [00:00<?, ? examples/s]

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [18]:
metric = evaluate.load("sacrebleu")

In [19]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [20]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [21]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint) # LOAD WEIGHTS

Metal device set to: Apple M1 Max


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [32]:
model

In [42]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator,
)

In [43]:
tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=32,
    collate_fn=data_collator,
)

In [44]:
#model.compile()

In [45]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [46]:
model

In [47]:
## Future, create validation dataset

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [48]:
push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_opus_books_model_2",
    tokenizer=tokenizer,
)

/Users/gema/Documents/The-Lord-of-The-Words-The-two-frameworks/notebooks/my_awesome_opus_books_model_2 is already a clone of https://huggingface.co/SoyGema/my_awesome_opus_books_model_2. Make sure you pull the latest changes with `repo.git_pull()`.


In [49]:
callbacks = [metric_callback, push_to_hub_callback]

In [50]:
### Habilitate GPU in keras, not only in Tensorflow or see the tutorial in TensorFlow and not keras

In [51]:

with tf.device('/GPU'):
    model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)

35/35 [==============================] - 271s 7s/step - loss: 4.1024 - val_loss: 3.6018


In [ ]:
#Less data require less training . Still not GPU MAC enabled.
# Putting a larger batch size might work as well 
# trying to compute acc as well ?

#model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1, callbacks=callbacks)

In [52]:
text = "translate English to portuguese: Hello. My Name is Maria. And I live in Oporto."

In [ ]:
from transformers import pipeline

In [ ]:
translator = pipeline("translation", model="my_awesome_opus_books_model_2")

In [ ]:
translator(text)

In [53]:
##Upload model to Hub

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("my_awesome_opus_books_model_2")
inputs = tokenizer(text, return_tensors="tf").input_ids

OSError: my_awesome_opus_books_model_2 does not appear to have a file named config.json. Checkout 'https://huggingface.co/my_awesome_opus_books_model_2/None' for available files.